# Get started with Model Registry

This notebook demonstrates how to use Evidently to:
* Generate a model performance report and calculate associated metrics.
* Log model metrics to MLFlow.
* Store the model in MLFlow as an artifact.
* Store the model performance report in MLFlow as an artifact.

In [ ]:
%load_ext autoreload
%autoreload 2

import joblib
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
import pandas as pd
from pathlib import Path
from sklearn import ensemble, model_selection
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

## Load Data

More information about the dataset can be found in UCI machine learning repository: https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

Acknowledgement: Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg

In [ ]:
# Download original dataset with: python src/pipelines/load_data.py 

raw_data = pd.read_csv(f"../data/raw_data.csv")
raw_data.head()

## Define column mapping

In [ ]:
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'mnth', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday', ]

In [ ]:
sample_data = raw_data.set_index('dteday').loc['2011-01-01 00:00:00':'2011-01-28 23:00:00'].reset_index()

print(sample_data.shape)

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    sample_data[numerical_features + categorical_features],
    sample_data[target],
    test_size=0.3
)

print(X_train.shape)
print(X_test.shape)

## Train a  Linear Regression Model

In [ ]:
model_lr = LinearRegression()
model_lr.fit(X_train, y_train) 

model_lr_path = Path('../models/model_lr.joblib')
joblib.dump(model_lr, model_lr_path)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

preds_lr = model_lr.predict(X_test)

me = mean_squared_error(y_test, preds_lr)
mae = mean_absolute_error(y_test, preds_lr)

print(me, mae)

## Train a RandomForestRegressor Model

In [ ]:
model_rf = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)
model_rf.fit(X_train, y_train) 

model_path = Path('../models/model_rf.joblib')
joblib.dump(model_rf, model_path)

In [ ]:

preds_rf = model_rf.predict(X_test)

me = mean_squared_error(y_test, preds_rf)
mae = mean_absolute_error(y_test, preds_rf)

print(me, mae)

# Model Registry

## Set up MLFlow

In [ ]:
# Set up MLFlow 
MLFLOW_TRACKING_URI = "http://localhost:5000"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

# Set up MLFlow Client
client = MlflowClient()
print(f"Client tracking uri: {client.tracking_uri}")

# Set experiment name
mlflow.set_experiment('5-Model-Registy')


## Registering a Model

- Docs on [mlflow.sklearn.log_model](https://www.mlflow.org/docs/latest/python_api/mlflow.sklearn.html?highlight=save_model#mlflow.sklearn.log_model)

### Log the `model_lr` model

In [ ]:
with mlflow.start_run() as run: 
    
    # Log the sklearn model and register as version 1
    mlflow.sklearn.log_model(
        sk_model=model_lr,
        artifact_path="LinearRegression"
    )

### Log and Register the `model_rf` model (3 times)

- Use `registered_model_name` to register a model automatically.
- If a registered model with the name doesn’t exist, the method registers a new model and creates `Version 1`.
- If a registered model with the name exists, the method creates a new model version.

INSTRUCTION: 
- Run the cell below 3 time to register 3 versions of the "bike-sharing-RandomForestRegressor" model

In [ ]:
from mlflow.models import infer_signature

with mlflow.start_run() as run: 

    # Show newly created run metadata info
    print("Experiment id: {}".format(run.info.experiment_id))
    print("Run id: {}".format(run.info.run_id))
    print("Run name: {}".format(run.info.run_name))
    print('MLFlow tracking uri:', mlflow.get_tracking_uri())
    print('MLFlow artifact uri:', mlflow.get_artifact_uri())
    run_id = run.info.run_id

    # Infer the model signature
    signature = infer_signature(y_test, preds_rf)

    # Log the sklearn model and register as version 1
    mlflow.sklearn.log_model(
        sk_model=model_rf,
        artifact_path="RandomForest",
        signature=signature,
        registered_model_name="RandomForest",
    )

In [ ]:
# Adding or Updating an MLflow Model Descriptions

client.update_model_version(
    name="RandomForest",
    version=3,
    description="This a model version 3 description added with update_model_version() method",
)

# Note: if you got an error, make sure you run the previous cell 3 times and save 3 versions of the model! 

## Discover models and their stages

In [ ]:
from pprint import pprint

# Listing and Searching MLflow Models

for rm in client.search_registered_models():
    pprint(dict(rm), indent=4)

In [ ]:
# Search for a specific model name and list its version details 

for mv in client.search_model_versions("name='RandomForest'"):
    pprint(dict(mv), indent=4)

## Transitioninig a model stage

In [ ]:
# Over the course of the model’s lifecycle, a model evolves—from development to staging to production. 
# You can transition a registered model to one of the stages: Staging, Production or Archived.

client.transition_model_version_stage(
    name="RandomForest", version=3, stage="Production"
)

## Download and use models from the registry

In [ ]:
model_version_uri = "models:/example-model@Champion"

In [ ]:
# Load the model from the model registry and score
model_uri = f"models:/RandomForest/3"
loaded_model = mlflow.sklearn.load_model(model_uri)
loaded_model

In [ ]:
loaded_model.predict(X_test)

## Deregistering, Deleting and Archiving models 

In [ ]:
# Archive models version 3 from Production into Archived

client = MlflowClient()
client.transition_model_version_stage(
    name="RandomForest", version=2, stage="Archived"
)

In [ ]:
# Note: Deleting registered models or model versions is irrevocable, so use it judiciously.

# Delete version 1 of the model
client.delete_model_version(
        name="RandomForest", version=1,
    )